In [1]:
import sys
import os 
import random
from random import randint
import numpy as np
import cv2
sys.path.append('..')
import Video_Utils
import CNN_Utils
import h5py
import time
import numpy as np
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Input, Convolution2D, UpSampling2D, MaxPooling2D, ZeroPadding2D, Reshape, merge
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.utils.np_utils import to_categorical
from keras.callbacks import ModelCheckpoint, TensorBoard
from keras.optimizers import RMSprop, SGD
from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input
from keras.models import Model
from keras.models import load_model
from keras import backend as K
from keras.layers.core import Lambda, Merge
from keras.engine import Layer

'''
From the command line you can convert a notebook to python with this command:

ipython nbconvert --to python <YourNotebook>.ipynb

You may have to install the python mistune package:

sudo pip install mistune
'''


Using TensorFlow backend.


'\nFrom the command line you can convert a notebook to python with this command:\n\nipython nbconvert --to python <YourNotebook>.ipynb\n\nYou may have to install the python mistune package:\n\nsudo pip install mistune\n'

In [2]:
############################################## GLOBAL VARIABLES #########################################
# These variables are specific to the dataset of the problem. 
# Our data is the following: 
#
# time-series of 4 cameras (128 x 128 x 3) looking at a robot complete a task (pick up object, place in bin)
# + corresponding time series of 7-d vector of joint commands (6 DOF + gripper) denoting the position delta command.
#
# In future comments, we will refer to the states and actions as x(t) and u(t) respectively, where 'state' 
# denotes the visual data sensed from the world (the 4 cameras) and action is the 7-d joint position delta vector. 
#
# Data was generated and collected using the V-REP (http://www.coppeliarobotics.com/) robot simulator 
# (free/education version). Please contact the author of this notebook for the specific scene and data-generation
# script 

global CAM_W
global CAM_H
global CAM_C
global NUM_CAMS
global ACTION_LEN

CAM_W = 128
CAM_H = 128
CAM_C = 3
NUM_CAMS = 4
ACTION_LEN = 7

In [5]:
####################################### MODEL PARAMETERS #########################################
global NUM_FUTURE_FRAMES
global NUM_PAST_FRAMES
#global NUM_VGG_FEAT_MAPS
#global VGG_FEAT_W
#global VGG_FEAT_H 
global GENC_1_FEAT_MAPS
global GENC_2_FEAT_MAPS
global GENC_3_FEAT_MAPS
global GENC_4_FEAT_MAPS

global GDEC_1_FEAT_MAPS
global GDEC_2_FEAT_MAPS
global GDEC_3_FEAT_MAPS
global GDEC_4_FEAT_MAPS

global CONVG5_FEAT_MAPS
global CONVG4_FEAT_MAPS
global CONVG3_FEAT_MAPS
global CONVG2_FEAT_MAPS
global CONVG1_FEAT_MAPS
global G_DENSE1
global G_DENSE2
global G_DENSE3
global GENERATOR_MODEL
global DISCRIMINATOR_MODEL

#VGG_FEAT_W = 64        # using "block1_pool"
#VGG_FEAT_H = 64        # using "block1_pool"  
#NUM_VGG_FEAT_MAPS = 64 # using "block1_pool" (per camera)
NUM_FUTURE_FRAMES = 10
NUM_PAST_FRAMES = 10
# Generator layers 
    # Part 1: Encoding for current frame
GENC_1_FEAT_MAPS = 64
GENC_2_FEAT_MAPS = 32
GENC_3_FEAT_MAPS = 16
GENC_4_FEAT_MAPS = 8
GDEC_4_FEAT_MAPS = CAM_C*NUM_CAMS
GDEC_3_FEAT_MAPS = round(GDEC_4_FEAT_MAPS * 2)
GDEC_2_FEAT_MAPS = round(GDEC_3_FEAT_MAPS * 2)
GDEC_1_FEAT_MAPS = round(GDEC_2_FEAT_MAPS * 2)
    # Part 2: DCGAN-type architecture
CONVG4_FEAT_MAPS = CAM_C*NUM_CAMS*NUM_FUTURE_FRAMES    # e.g. == 3*4*10 = 120
CONVG3_FEAT_MAPS = round(CONVG5_FEAT_MAPS * 2)       # e.g. == 240
CONVG2_FEAT_MAPS = round(CONVG4_FEAT_MAPS * 2)       # e.g. == 480
CONVG1_FEAT_MAPS = round(CONVG3_FEAT_MAPS * 2)       # e.g. == 960
 
G_DENSE1 = 1200 # the vector going into this dense layer will be on order of 512+63 ~ 600-dim 
G_DENSE2 = 4096


In [4]:
################################################# THE MODEL ####################################################
'''
An explanation of the dynamics model and the buffers: 

We are training a model to produce p(x(t+1: t+F) | x(t), u(t: t+F-1)) 

I.e.: predict the future F frames, 
      given the current frame, future F-1 actions to take
      
The GENERATOR_MODEL will be responsible for producing a tensor of shape: (F, W, H, 3*NUM_CAMS)

The DISCRIMINATOR_MODEL will output a single scalar (e.g. in (-1, 1) range) after taking in the above ^ shape 
as input, + the proposed actions, and determine whether the frame-action sequence was real or generated. 

I.e.: Discriminator estimates p(sequence_real | x(t), x(t+1: t+F), u(t: t+F-1))

'''
# Can use part of pre-trained VGG model to seed features with reasonable features: (used online during training)  
#vgg_preprocessor = CNN_Utils.GetVGGModel("block1_pool", CAM_W, CAM_H, print_timing=1) 
# ^ Note: not using for this experiment (creates too many feature maps per camera for current hardware)

# Gen Model input #1: current frame
input_curr_frame = Input(shape=(CAM_W, CAM_H, CAM_C*NUM_CAMS), name='input_curr_frame')
# Step 1: Compress and encode to relevant vectorized information -
    # Idea: try regularizing by making and training a network branch that must be able 
    # to reconstruct the input frame from the encoding
    # input_curr_frame sample shape: 128x128x12 (3channels * 4cameras)
frame_enc = Convolution2D(GENC_1_FEAT_MAPS, 9, 9, strides=(2, 2), activation='elu', border_mode='same', name='genc_1')(input_curr_frame)
frame_enc = MaxPooling2D((2, 2), border_mode='same')(frame_enc) #e.g. 64x64x64 (W,H,filters)
frame_enc = Convolution2D(GENC_2_FEAT_MAPS, 7, 7, activation='elu', border_mode='same', name='genc_2')(frame_enc)
frame_enc = MaxPooling2D((2, 2), border_mode='same')(frame_enc) #e.g. 32x32x32
frame_enc = Convolution2D(GENC_3_FEAT_MAPS, 5, 5, activation='elu', border_mode='same', name='genc_3')(frame_enc)
frame_enc = MaxPooling2D((2, 2), border_mode='same')(frame_enc) #e.g. 16x16x16
frame_enc = Convolution2D(GENC_4_FEAT_MAPS, 3, 3, activation='elu', border_mode='same', name='genc_4')(frame_enc)
frame_enc = MaxPooling2D((2, 2), border_mode='same')(frame_enc) #e.g. 8x8x8
frame_enc_flat = Flatten()(frame_enc_flat, name='genc_flat') #e.g. 512-d vector --> for Generator usage
    # Decoder-as-a-regularizer: 
frame_dec = Convolution2D(GDEC_1_FEAT_MAPS, 3, 3, activation='elu', border_mode='same', name='gdec_1')(frame_enc)
frame_dec = UpSampling2D(size=(2, 2))(frame_dec) # e.g. 16x16x96
frame_dec = Convolution2D(GDEC_2_FEAT_MAPS, 5, 5, activation='elu', border_mode='same', name='gdec_2')(frame_dec)
frame_dec = UpSampling2D(size=(2, 2))(frame_dec) # e.g. 32x32x48
frame_dec = Convolution2D(GDEC_3_FEAT_MAPS, 5, 5, activation='elu', border_mode='same', name='gdec_2')(frame_dec)
frame_dec = UpSampling2D(size=(2, 2))(frame_dec) # e.g. 64x64x24
frame_dec = Convolution2D(GDEC_4_FEAT_MAPS, 5, 5, activation='elu', border_mode='same', name='gdec_2')(frame_dec)
frame_dec = UpSampling2D(size=(2, 2))(frame_dec) # e.g. 128x128x12
frame_dec = Convolution2D(GDEC_4_FEAT_MAPS, 3, 3, activation='elu', border_mode='same', name='gdec_2')(frame_dec)
# Gen Model input #2: future actions
input_future_actions_raw = Input(shape=((NUM_FUTURE_FRAMES-1)*ACTION_LEN,), name='input_future_actions_raw')
# Step 2: Merge inputs, project, reshape
gen_merged_inputs = merge([frame_enc_flat, input_future_actions_raw], mode='concat', concat_axis=1, name='merged_gen_inputs')
G_A1 = Dense(output_dim=G_DENSE1, activation='elu', name='G_A1')(gen_merged_inputs)
G_A2 = Dense(output_dim=G_DENSE2, activation='elu', name='G_A2')(G_A1)

gen_out = Reshape((4, 4, round(G_DENSE2/(4*4)), name='gout_1')(G_A2) # e.g. 4x4x256
gen_out = UpSampling2D(size=(2, 2))(gen_out) # e.g. 8x8x256
gen_out = Convolution2D(CONVG1_FEAT_MAPS, 3, 3, activation='elu', border_mode='same', name='gout_2')(gen_out)
gen_out = UpSampling2D(size=(2, 2))(gen_out) # e.g. 16x16x960 
gen_out = Convolution2D(CONVG2_FEAT_MAPS, 5, 5, activation='elu', border_mode='same', name='gout_3')(gen_out)
gen_out = UpSampling2D(size=(2, 2))(gen_out) # e.g. 32x32x480
gen_out = Convolution2D(CONVG3_FEAT_MAPS, 5, 5, activation='elu', border_mode='same', name='gout_4')(gen_out)
gen_out = UpSampling2D(size=(2, 2))(gen_out) # e.g. 64x64x240
gen_out = Convolution2D(CONVG4_FEAT_MAPS, 7, 7, activation='elu', border_mode='same', name='gout_5')(gen_out)
gen_out = UpSampling2D(size=(2, 2))(gen_out) # e.g. 128x128x120
gen_out = Convolution2D(CONVG4_FEAT_MAPS, 3, 3, activation='elu', border_mode='same', name='gout_final')(gen_out)
                  
                  
# Define the full model structure: 
gen_model_inputs = [input_prev_frames_raw, input_prev_actions_raw, input_future_actions_raw]
# Final output shape: W, H, 3*NUM_CAMS*NUM_FUTURE_FRAMES, e.g. 128x128x120
gen_model_outputs = [gen_out] 


# Now merge with information about future actions: 
merged_input = merge([merged_input, future_action_branch], 
                      mode='concat', concat_axis=3, name='merged_efficient_inputs_all')
# ^ This will have size [W, H, NUM_PAST_FRAMES + 1], e.g. [128, 128, 11]

# Conv layers for predicting the next frames given all relevant data: 
Out_C1 = Convolution2D(CONVF1_FEAT_MAPS, 7, 7, activation='elu', border_mode='same', name='Out_C1')(merged_input)
Out_C2 = Convolution2D(CONVF2_FEAT_MAPS, 3, 3, activation='elu', border_mode='same', name='Out_C2')(Out_C1)
Out_C3 = Convolution2D(CONVF3_FEAT_MAPS, 3, 3, activation='elu', border_mode='same', name='Out_C3')(Out_C2)
Out_C4 = Convolution2D(CONVF4_FEAT_MAPS, 3, 3, activation='elu', border_mode='same', name='Out_C4')(Out_C3)
# ^ Out_C4 is a final output layer, spits out a frame (defined as WxHx3*num_cams*num_future_frames)

# Define the full model structure: 
model_inputs = [input_prev_frames_raw, input_prev_actions_raw, input_future_actions_raw]
# Review of input dimensions: (caller of train/predict must expand_dims to achieve shapes)
# input_prev_frames_raw: [W, H, 3*NUM_CAMS*NUM_PAST_FRAMES] e.g. [1, 128, 128, 120]
# input_prev_actions_raw: [NUM_PAST_FRAMES*ACTION_LEN,] e.g. [1, 70] 
# input_future_actions_raw: [(NUM_FUTURE_FRAMES-1)*ACTION_LEN,] e.g. [1, 63]
model_outputs = [Out_C4] 
# Target output data: 
# Out_C4: [W, H, 3*NUM_CAMS*NUM_FUTURE_FRAMES], e.g. [1, 128, 128, 120]
UDM_model = Model(input=model_inputs, output=model_outputs)

UDM_optimizer = RMSprop(lr=0.001, rho=0.9, epsilon=1e-08, decay=0.0)
UDM_model.compile(loss='binary_crossentropy', optimizer=UDM_optimizer) 
# ^ TODO: custom loss function more appropriate for sequence of similar images...   

UDM_model.summary()

NameError: name 'Input' is not defined

In [ ]:
####################################### INITIALIZATION, SETUP OF MODEL #########################################

In [ ]:
########################################## Globals for training params ####################################

global TOTAL_TRAINING_ITRS
global SAVE_CHECKPOINT_ITRS
global NUM_DEMONSTRATIONS
global CURR_DEMONSTRATION
global LENGTH_CURR_DEMONSTRATION # e.g. current demo we're looking at is 230 timesteps
global T_CURR_DEMONSTRATION      # and e.g. we're currently on timestep 87
global PERCENT_TRAIN             # percent of data used for training vs. valudation
global DEMONSTRATION_FOLDERS
global TRAINING_FOLDERS
global TESTING_FOLDERS
global NUM_TRAINING_DEMONSTRATIONS
global NUM_TESTING_DEMONSTRATIONS
global IMAGE_COMPARE_CHECKPOINT

global PREV_FRAMES_BUFFER
global PREV_ACTION_BUFFER
global FUTURE_FRAMES_BUFFER
global FUTURE_ACTION_BUFFER
global MODEL_LOSS_BUFFER

PREV_FRAMES_BUFFER = np.zeros((NUM_PAST_FRAMES, CAM_W, CAM_H, CAM_C))
PREV_ACTION_BUFFER = np.zeros((NUM_PAST_FRAMES, ACTION_LEN))
FUTURE_FRAMES_BUFFER = np.zeros((NUM_FUTURE_FRAMES, CAM_W, CAM_H, CAM_C))
FUTURE_ACTION_BUFFER = np.zeros((NUM_FUTURE_FRAMES-1, ACTION_LEN))

TOTAL_TRAINING_ITRS = 100000
SAVE_CHECKPOINT_ITRS = 100
IMAGE_COMPARE_CHECKPOINT = 5
CURR_DEMONSTRATION = -1 # start on the first one, loop to another one each itr
LENGTH_CURR_DEMONSTRATION = -1
PERCENT_TRAIN = 0.75
MODEL_LOSS_BUFFER = np.zeros((10,1))

In [ ]:
########################################## START THE TRAINING #######################################

print("Running program...")

DEMONSTRATION_FOLDERS = Video_Utils.GetFoldersForRuns()

NUM_DEMONSTRATIONS = len(DEMONSTRATION_FOLDERS)

for f in DEMONSTRATION_FOLDERS:
    print(f)

# Separate into training and testing data based on the number of recordings (folders) we have: 
random.shuffle(DEMONSTRATION_FOLDERS)    
lim_separate = round(NUM_DEMONSTRATIONS * PERCENT_TRAIN)    
TRAINING_FOLDERS = DEMONSTRATION_FOLDERS[0:lim_separate]
TESTING_FOLDERS = DEMONSTRATION_FOLDERS[lim_separate:]
NUM_TRAINING_DEMONSTRATIONS = len(TRAINING_FOLDERS)
NUM_TESTING_DEMONSTRATIONS = len(TESTING_FOLDERS)

print(NUM_TRAINING_DEMONSTRATIONS); print(NUM_TESTING_DEMONSTRATIONS); print(NUM_DEMONSTRATIONS) # a + b = c

    

# Data flow process: we train on entire folder (sample run of a robot) before moving on to the next to 
# amortize the time required to load that folder's training data into RAM (multiple seconds). For a dynamics
# model this should be perfectly acceptable because the dynamics to be learned are ideally the *same* between
# different sample runs recorded on the (simulated) robot. 
itrs = 0
while itrs < TOTAL_TRAINING_ITRS:
    
    # Choose which expert demonstration we're using: 
    CURR_DEMONSTRATION = randint(0,NUM_TRAINING_DEMONSTRATIONS-1)

    frames, actions = Video_Utils.LoadFramesActionsFromFolder(TRAINING_FOLDERS[CURR_DEMONSTRATION], CAM_W, CAM_H, CAM_C*NUM_CAMS, ACTION_LEN)
    # ^ about 5 secs
    
    print(frames.shape)
    print(actions.shape)
    
    LENGTH_CURR_DEMONSTRATION = frames.shape[0] # number of timesteps for this demonstration
    
    # New demonstration, reset the relevant data buffers:
    PREV_FRAMES_BUFFER = np.zeros((NUM_PAST_FRAMES, CAM_W, CAM_H, CAM_C*NUM_CAMS))
    PREV_ACTION_BUFFER = np.zeros((NUM_PAST_FRAMES, ACTION_LEN))
    FUTURE_FRAMES_BUFFER = np.zeros((NUM_FUTURE_FRAMES, CAM_W, CAM_H, CAM_C*NUM_CAMS))
    FUTURE_ACTION_BUFFER = np.zeros((NUM_FUTURE_FRAMES-1, ACTION_LEN))
    
    print("\n===== Training on robot sample run #:"+str(CURR_DEMONSTRATION)+" with num timesteps: "+str(LENGTH_CURR_DEMONSTRATION)+"\n")
    
    for i in range(0, round(LENGTH_CURR_DEMONSTRATION/NUM_FUTURE_FRAMES)): 
        # only sample a limited number from each demo before moving on to next demo. 
        t = randint(0,LENGTH_CURR_DEMONSTRATION-1)	# now we have random shuffle training within a demo.
            
        ##### STEP 1: Load Buffers with Data #####    
        ms1 = time.time()*1000.0 
        # Past data: (up to and including x(t), and u(t) - which causes x(t+1), etc)
        if (t >= NUM_PAST_FRAMES): # regular, in-bounds
            PREV_FRAMES_BUFFER = frames[t-NUM_PAST_FRAMES:t, :] 
            PREV_ACTION_BUFFER_BUFFER = actions[t-NUM_PAST_FRAMES:t, :]
        else: # t is less than past frames, need to concat zeros at beginning
            LIM1 = np.abs(t - NUM_PAST_FRAMES) 
            PREV_FRAMES_BUFFER = np.concatenate((np.zeros((LIM1, CAM_W, CAM_H, CAM_C*NUM_CAMS)), frames[0:t, :]), axis=0) 
            PREV_ACTIONS_BUFFER = np.concatenate((np.zeros((LIM1, ACTION_LEN)), actions[0:t, :]), axis=0)
        # Future Data: (include 1 less future action than frame to retain logical p(x(t+1)|x(t),u(t)) structure)
        if (t <= LENGTH_CURR_DEMONSTRATION - NUM_FUTURE_FRAMES - 1):
            FUTURE_FRAMES_BUFFER = frames[t:t+NUM_FUTURE_FRAMES, :]
            FUTURE_ACTIONS_BUFFER = actions[t:t+NUM_FUTURE_FRAMES-1, :]
        else: # running off the end
            LIM2 = t + NUM_FUTURE_FRAMES - LENGTH_CURR_DEMONSTRATION 
            # ^ We need to repeat last frame this many times / add this many zero actions
            FUTURE_FRAMES_BUFFER = frames[t:LENGTH_CURR_DEMONSTRATION, :]
            last_frame = np.expand_dims(frames[LENGTH_CURR_DEMONSTRATION-1], axis=0)
            for j in range(0,LIM2):
                FUTURE_FRAMES_BUFFER = np.concatenate((FUTURE_FRAMES_BUFFER, last_frame), axis=0) 
            FUTURE_ACTIONS_BUFFER = np.concatenate((actions[t:LENGTH_CURR_DEMONSTRATION, :], np.zeros((LIM2, ACTION_LEN))), axis=0)
        ms2 = time.time()*1000.0
        print("\nLoaded data for timestep "+str(t)+" in "+str(round((ms2-ms1)/1, 3))+" msecs\n")
        
        #print(PREV_FRAMES_BUFFER.shape)
        #print(PREV_ACTION_BUFFER.shape)
        #print(FUTURE_FRAMES_BUFFER.shape)
        #print(FUTURE_ACTION_BUFFER.shape)
        
        ##### STEP 2: Train on Batch of Data Gathered Above #####
        ms1 = time.time()*1000.0
        # inputs: [input_prev_frames_raw, input_prev_actions_raw, input_future_actions_raw]
        # outputs/training targets: [future_frames]
        model_loss = UDM_model.train_on_batch(
                     [np.expand_dims(PREV_FRAMES_BUFFER.reshape(CAM_W, CAM_H, CAM_C*NUM_CAMS*NUM_PAST_FRAMES), axis=0), 
                      np.expand_dims(PREV_ACTION_BUFFER.reshape(-1), axis=0), 
                      np.expand_dims(FUTURE_ACTION_BUFFER.reshape(-1), axis=0)], # <--- inputs 
                     [np.expand_dims(FUTURE_FRAMES_BUFFER.reshape(CAM_W, CAM_H, CAM_C*NUM_CAMS*NUM_PAST_FRAMES), axis=0)])
        
        ms2 = time.time()*1000.0
        print("===================================================")
        print("iter: " + str(itrs) + " / timestep: "+str(t))
        print("Model update time: "+str(round((ms2-ms1)/1000, 3)) + ' secs')
        print("Current Model Loss: "+str(model_loss))
        MODEL_LOSS_BUFFER = np.roll(MODEL_LOSS_BUFFER, 1); MODEL_LOSS_BUFFER[0] = model_loss
        print(MODEL_LOSS_BUFFER); print("\n")
        
        ##### STEP 3: CHECKPOINTS #####
        
        if (itrs % IMAGE_COMPARE_CHECKPOINT == 0 ): # View how the model is doing
            print("Image compare checkpoint...")
            timestamp = str(time.time()).replace(".","")
            
            # Get the generated future frames: 
            ms1 = time.time()*1000.0
            GEN_FUTURE_FRAMES = UDM_model.predict(
                                [np.expand_dims(PREV_FRAMES_BUFFER.reshape(CAM_W, CAM_H, CAM_C*NUM_CAMS*NUM_PAST_FRAMES), axis=0), 
                                 np.expand_dims(PREV_ACTION_BUFFER.reshape(-1), axis=0), 
                                 np.expand_dims(FUTURE_ACTION_BUFFER.reshape(-1), axis=0)])
            GEN_FUTURE_FRAMES = GEN_FUTURE_FRAMES[0]
            ms2 = time.time()*1000.0
            print("Model prediction time: "+str(round((ms2-ms1)/1000, 3)) +' secs'+' for '+str(NUM_FUTURE_FRAMES)+' future frames.')
            
            imgs_gen = Video_Utils.ViewFutureFrames(GEN_FUTURE_FRAMES.reshape(NUM_FUTURE_FRAMES, CAM_W, CAM_H, CAM_C*NUM_CAMS))
            imgs_split = np.zeros((30, NUM_FUTURE_FRAMES*CAM_W, 3)) # goes in between to separate real/generated images
            imgs_gt = Video_Utils.ViewFutureFrames(FUTURE_FRAMES_BUFFER)
            #print(imgs_gt.shape); print(imgs_split.shape); print(imgs_gen.shape);
            imgs_compare = np.concatenate((imgs_gt, imgs_split, imgs_gen),axis=0)
            #print(imgs_compare.shape)
            #cv2.imshow('image',imgs_compare) 
            #cv2.waitKey(0)
            img_filename = 'sample_'+timestamp+'.png' 
            cv2.imwrite(img_filename,(imgs_compare*255 + 127.5))
            print("Wrote new image sample checkpoint at: "+img_filename)
            
        if (itrs % SAVE_CHECKPOINT_ITRS == 0): # save progress
            print("Model save checkpoint, itr: "+str(itrs))
            ms1 = time.time()*1000.0
            timestamp = str(time.time()).replace(".","")
            mean_recent_loss = round(np.mean(MODEL_LOSS_BUFFER), 4)
            model_str_name = 'UDM_weights_'+str(mean_recent_loss)+'.h5' # pro-tip: manually re-name after each run... 
            UDM_model.save(model_str_name)
            ms2 = time.time()*1000.0
            print("Model save time: "+str(round((ms2-ms1)/1000, 3)) +' secs')

        itrs = itrs + 1 # don't forget to increment total training itrs counter    

Running program...
u_sequence_1486408000621
u_sequence_1486420877768
u_sequence_1486421372155
u_sequence_1486422003939
u_sequence_1486423780289
u_sequence_1486424226636
u_sequence_1486424664609
u_sequence_1486425113072
u_sequence_1486425723665
u_sequence_1486426394928
u_sequence_1486483126414
u_sequence_1486483372904
u_sequence_1486483607589
u_sequence_1486483842933
u_sequence_1486484146973
u_sequence_1486484663068
u_sequence_1486485509467
u_sequence_1486485832231
u_sequence_1486486076841
u_sequence_1486486803219
15
5
20
(185, 128, 128, 12)
(185, 7)

===== Training on robot sample run #:13 with num timesteps: 185


Loaded data for timestep 8 in 24.032 msecs

iter: 0 / timestep: 8
Model update time: 15.453 secs
Current Model Loss: -0.436337
[[-0.43633705]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]]


Image compare checkpoint...
Model prediction time: 1.566 secs for 10 future frames.
Wrote new ima

In [5]:
################## make a video... #############

vid_data = Video_Utils.GetDataFromFolder('old_visuals', do_normalize=0, w=1280, h=1054)
print(vid_data.shape)

(1054, 1280, 3)


ValueError: could not broadcast input array from shape (1054,1280,3) into shape (1280,1054,3)